# Comparing RL Algorithms: Choosing the Right Tool

Welcome to the final notebook of the Classic Algorithms section! Let's put everything together and understand when to use each algorithm.

## What You'll Learn

By the end of this notebook, you'll understand:
- How Monte Carlo, TD, Q-Learning, and SARSA compare
- Learning speed, variance, and convergence differences
- When each algorithm shines (with a toolbox analogy!)
- How to choose the right algorithm for your problem
- Empirical analysis on multiple environments

**Prerequisites:** Notebooks 1-4 (MC, TD, Q-Learning, SARSA)

**Time:** ~30 minutes

---
## The Big Picture: The RL Toolbox

Think of these algorithms as tools in a toolbox - each is best for certain situations:

```
    ┌────────────────────────────────────────────────────────────────┐
    │                 THE RL ALGORITHM TOOLBOX                       │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  🔨 MONTE CARLO = The Careful Craftsman                        │
    │     "I wait until the job is COMPLETELY done before I judge."  │
    │     • Waits for full episode                                  │
    │     • Unbiased but slow                                       │
    │     • High variance                                           │
    │     Best for: Card games, episodic tasks with clear endings   │
    │                                                                │
    │  ⚡ TD LEARNING = The Quick Learner                            │
    │     "I update my estimates as I go - no need to wait!"        │
    │     • Updates every step                                      │
    │     • Some bias but faster                                    │
    │     • Lower variance                                          │
    │     Best for: Continuous tasks, faster learning               │
    │                                                                │
    │  🎯 Q-LEARNING = The Optimist                                 │
    │     "I assume I'll always take the BEST action in the future."│
    │     • Off-policy: learns optimal regardless of behavior       │
    │     • Can be risky during exploration                         │
    │     Best for: Games, simulations where mistakes are cheap     │
    │                                                                │
    │  🛡️ SARSA = The Realist                                       │
    │     "I account for my OWN imperfect behavior."                │
    │     • On-policy: learns value of actual behavior              │
    │     • Safer during exploration                                │
    │     Best for: Robotics, trading, safety-critical systems      │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch, Rectangle, Circle
from collections import defaultdict
import time

# Visualize the algorithm characteristics
fig, ax = plt.subplots(figsize=(14, 8))
ax.set_xlim(0, 14)
ax.set_ylim(0, 10)
ax.axis('off')

ax.text(7, 9.5, 'The Four Classic RL Algorithms', ha='center', fontsize=18, fontweight='bold')

algorithms = [
    {
        'name': 'Monte Carlo',
        'icon': '🔨',
        'color': '#bbdefb',
        'edge': '#1976d2',
        'traits': ['Unbiased', 'High variance', 'Episode-based'],
        'x': 1.5
    },
    {
        'name': 'TD Learning',
        'icon': '⚡',
        'color': '#fff3e0',
        'edge': '#f57c00',
        'traits': ['Bootstraps', 'Low variance', 'Step-based'],
        'x': 4.5
    },
    {
        'name': 'Q-Learning',
        'icon': '🎯',
        'color': '#c8e6c9',
        'edge': '#388e3c',
        'traits': ['Off-policy', 'Finds optimal', 'Bold'],
        'x': 7.5
    },
    {
        'name': 'SARSA',
        'icon': '🛡️',
        'color': '#f3e5f5',
        'edge': '#7b1fa2',
        'traits': ['On-policy', 'Safe', 'Cautious'],
        'x': 10.5
    }
]

for algo in algorithms:
    # Main box
    box = FancyBboxPatch((algo['x'], 3), 2.5, 5, boxstyle="round,pad=0.1",
                          facecolor=algo['color'], edgecolor=algo['edge'], linewidth=3)
    ax.add_patch(box)
    
    # Icon and name
    ax.text(algo['x'] + 1.25, 7.2, algo['icon'], ha='center', fontsize=20)
    ax.text(algo['x'] + 1.25, 6.5, algo['name'], ha='center', fontsize=11, fontweight='bold')
    
    # Traits
    for i, trait in enumerate(algo['traits']):
        ax.text(algo['x'] + 1.25, 5.5 - i*0.7, f'• {trait}', ha='center', fontsize=9)

# Arrows showing relationships
ax.annotate('', xy=(4.4, 5.5), xytext=(3.6, 5.5),
            arrowprops=dict(arrowstyle='<->', lw=2, color='#666'))
ax.text(4, 5.9, 'bias-variance\ntradeoff', ha='center', fontsize=8, color='#666')

ax.annotate('', xy=(10.4, 5.5), xytext=(9.6, 5.5),
            arrowprops=dict(arrowstyle='<->', lw=2, color='#666'))
ax.text(10, 5.9, 'safety-optimality\ntradeoff', ha='center', fontsize=8, color='#666')

# Summary at bottom
ax.text(7, 1.5, 'All are model-free, tabular methods for learning value functions', 
        ha='center', fontsize=12, style='italic', color='#666')
ax.text(7, 0.8, 'The right choice depends on your specific problem!', 
        ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

---
## Setting Up the Environments

We'll compare algorithms on two environments:
1. **GridWorld** - Standard navigation task
2. **CliffWalking** - Highlights safety differences

In [ ]:
class GridWorld:
    """
    Standard 4x4 Grid World.
    Goal: Navigate from (0,0) to (3,3)
    """
    
    def __init__(self):
        self.size = 4
        self.goal = (3, 3)
        self.reset()
    
    def reset(self):
        self.pos = (0, 0)
        return self.pos
    
    def step(self, action):
        row, col = self.pos
        if action == 0: row = max(0, row - 1)      # UP
        elif action == 1: col = min(3, col + 1)   # RIGHT
        elif action == 2: row = min(3, row + 1)   # DOWN
        elif action == 3: col = max(0, col - 1)   # LEFT
        
        self.pos = (row, col)
        done = self.pos == self.goal
        reward = 10 if done else -1
        return self.pos, reward, done


class CliffWalking:
    """
    Cliff Walking environment.
    Falling off the cliff gives -100 and resets to start.
    """
    
    def __init__(self):
        self.height = 4
        self.width = 12
        self.start = (3, 0)
        self.goal = (3, 11)
        self.cliff = [(3, i) for i in range(1, 11)]
        self.reset()
    
    def reset(self):
        self.pos = self.start
        return self.pos
    
    def step(self, action):
        row, col = self.pos
        if action == 0: row = max(0, row - 1)
        elif action == 1: col = min(self.width - 1, col + 1)
        elif action == 2: row = min(self.height - 1, row + 1)
        elif action == 3: col = max(0, col - 1)
        
        self.pos = (row, col)
        
        if self.pos in self.cliff:
            self.pos = self.start
            return self.pos, -100, False
        
        if self.pos == self.goal:
            return self.pos, -1, True
        
        return self.pos, -1, False


print("Environments created!")
print("  • GridWorld: 4x4 grid, -1 per step, +10 at goal")
print("  • CliffWalking: 4x12 grid with cliff, -100 if fall")

---
## Implementing All Algorithms

Let's implement all four algorithms with consistent interfaces for fair comparison.

In [ ]:
def monte_carlo_control(env, n_episodes, gamma=0.9, epsilon=0.1):
    """
    Monte Carlo Control with ε-greedy exploration.
    
    KEY: Updates only at END of each episode.
    """
    Q = defaultdict(lambda: np.zeros(4))
    returns = defaultdict(list)
    rewards_history = []
    
    def policy(state):
        if np.random.random() < epsilon:
            return np.random.randint(0, 4)
        return np.argmax(Q[state])
    
    for ep in range(n_episodes):
        # Generate complete episode
        episode = []
        state = env.reset()
        total_reward = 0
        
        for _ in range(200):
            action = policy(state)
            next_state, reward, done = env.step(action)
            episode.append((state, action, reward))
            total_reward += reward
            state = next_state
            if done:
                break
        
        # Update Q-values from complete episode
        G = 0
        visited = set()
        for t in range(len(episode) - 1, -1, -1):
            s, a, r = episode[t]
            G = gamma * G + r
            if (s, a) not in visited:
                visited.add((s, a))
                returns[(s, a)].append(G)
                Q[s][a] = np.mean(returns[(s, a)])
        
        rewards_history.append(total_reward)
    
    return Q, rewards_history


def td_prediction(env, n_episodes, alpha=0.1, gamma=0.9, epsilon=0.1):
    """
    TD(0) with Q-values (essentially same as Q-learning for control).
    
    KEY: Updates EVERY step using bootstrap.
    """
    Q = defaultdict(lambda: np.zeros(4))
    rewards_history = []
    
    for ep in range(n_episodes):
        state = env.reset()
        total_reward = 0
        
        for _ in range(200):
            # ε-greedy action selection
            if np.random.random() < epsilon:
                action = np.random.randint(0, 4)
            else:
                action = np.argmax(Q[state])
            
            next_state, reward, done = env.step(action)
            total_reward += reward
            
            # TD update (same as Q-learning)
            td_target = reward + gamma * np.max(Q[next_state])
            Q[state][action] += alpha * (td_target - Q[state][action])
            
            state = next_state
            if done:
                break
        
        rewards_history.append(total_reward)
    
    return Q, rewards_history


def q_learning(env, n_episodes, alpha=0.1, gamma=0.9, epsilon=0.1):
    """
    Q-Learning: Off-policy TD control.
    
    KEY: Uses max Q(s', a') - learns optimal policy.
    """
    Q = defaultdict(lambda: np.zeros(4))
    rewards_history = []
    
    for ep in range(n_episodes):
        state = env.reset()
        total_reward = 0
        
        for _ in range(200):
            # ε-greedy behavior policy
            if np.random.random() < epsilon:
                action = np.random.randint(0, 4)
            else:
                action = np.argmax(Q[state])
            
            next_state, reward, done = env.step(action)
            total_reward += reward
            
            # Q-LEARNING UPDATE: max over next actions
            td_target = reward + gamma * np.max(Q[next_state])
            Q[state][action] += alpha * (td_target - Q[state][action])
            
            state = next_state
            if done:
                break
        
        rewards_history.append(total_reward)
    
    return Q, rewards_history


def sarsa(env, n_episodes, alpha=0.1, gamma=0.9, epsilon=0.1):
    """
    SARSA: On-policy TD control.
    
    KEY: Uses Q(s', a') where a' is actual next action.
    """
    Q = defaultdict(lambda: np.zeros(4))
    rewards_history = []
    
    def policy(state):
        if np.random.random() < epsilon:
            return np.random.randint(0, 4)
        return np.argmax(Q[state])
    
    for ep in range(n_episodes):
        state = env.reset()
        action = policy(state)
        total_reward = 0
        
        for _ in range(200):
            next_state, reward, done = env.step(action)
            next_action = policy(next_state)
            total_reward += reward
            
            # SARSA UPDATE: uses actual next action
            td_target = reward + gamma * Q[next_state][next_action]
            Q[state][action] += alpha * (td_target - Q[state][action])
            
            state = next_state
            action = next_action
            if done:
                break
        
        rewards_history.append(total_reward)
    
    return Q, rewards_history


print("All algorithms implemented!")
print("  • monte_carlo_control() - Episode-based learning")
print("  • td_prediction() - Step-based with bootstrapping")
print("  • q_learning() - Off-policy, finds optimal")
print("  • sarsa() - On-policy, safer exploration")

---
## Comparison 1: GridWorld Performance

Let's compare all algorithms on the standard GridWorld task.

In [ ]:
# Run comparison on GridWorld

n_episodes = 2000
n_runs = 10  # Average over multiple runs for reliable results

algorithms = {
    'Monte Carlo': monte_carlo_control,
    'Q-Learning': q_learning,
    'SARSA': sarsa
}

colors = {
    'Monte Carlo': '#1976d2',
    'Q-Learning': '#388e3c',
    'SARSA': '#7b1fa2'
}

results_grid = {name: [] for name in algorithms}

print("GRIDWORLD COMPARISON")
print("="*60)
print(f"Running {n_runs} trials of {n_episodes} episodes each...\n")

for name, algo in algorithms.items():
    print(f"  {name}...", end=" ", flush=True)
    start_time = time.time()
    
    for run in range(n_runs):
        env = GridWorld()
        _, rewards = algo(env, n_episodes)
        results_grid[name].append(rewards)
    
    elapsed = time.time() - start_time
    print(f"done ({elapsed:.1f}s)")

# Calculate means and standard deviations
means_grid = {name: np.mean(runs, axis=0) for name, runs in results_grid.items()}
stds_grid = {name: np.std(runs, axis=0) for name, runs in results_grid.items()}

print("\n" + "="*60)

In [ ]:
# Visualize GridWorld results

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Learning curves
ax1 = axes[0]
window = 50

for name, mean_rewards in means_grid.items():
    smoothed = np.convolve(mean_rewards, np.ones(window)/window, mode='valid')
    ax1.plot(range(window-1, len(mean_rewards)), smoothed, 
             label=name, color=colors[name], linewidth=2)

ax1.set_xlabel('Episode', fontsize=12)
ax1.set_ylabel('Total Reward (smoothed)', fontsize=12)
ax1.set_title('Learning Curves on GridWorld', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Right: Final performance boxplot
ax2 = axes[1]
final_results = [np.array(results_grid[name])[:, -100:].mean(axis=1) for name in algorithms]
bp = ax2.boxplot(final_results, labels=list(algorithms.keys()), patch_artist=True)

for patch, name in zip(bp['boxes'], algorithms.keys()):
    patch.set_facecolor(colors[name])
    patch.set_alpha(0.7)

ax2.set_ylabel('Average Reward (last 100 episodes)', fontsize=12)
ax2.set_title('Final Performance Distribution', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Print final statistics
print("\nFINAL PERFORMANCE (last 100 episodes):")
print("-"*50)
for name in algorithms:
    final_avg = np.mean(means_grid[name][-100:])
    final_std = np.std([np.mean(r[-100:]) for r in results_grid[name]])
    print(f"{name:15s}: {final_avg:.2f} ± {final_std:.2f}")

---
## Comparison 2: CliffWalking (Safety Test)

This environment highlights the SARSA vs Q-Learning difference!

In [ ]:
# Run comparison on CliffWalking

n_episodes = 500
n_runs = 10

# Only Q-Learning and SARSA for cliff (MC is slow)
cliff_algorithms = {
    'Q-Learning': q_learning,
    'SARSA': sarsa
}

results_cliff = {name: [] for name in cliff_algorithms}

print("CLIFF WALKING COMPARISON")
print("="*60)
print("This environment tests SAFETY during exploration!\n")

for name, algo in cliff_algorithms.items():
    print(f"  {name}...", end=" ", flush=True)
    for run in range(n_runs):
        env = CliffWalking()
        _, rewards = algo(env, n_episodes, alpha=0.5, gamma=1.0)
        results_cliff[name].append(rewards)
    print("done")

means_cliff = {name: np.mean(runs, axis=0) for name, runs in results_cliff.items()}

print("\n" + "="*60)

In [ ]:
# Visualize CliffWalking results

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Learning curves
ax1 = axes[0]
window = 20

for name, mean_rewards in means_cliff.items():
    ax1.plot(mean_rewards, alpha=0.3, color=colors[name])
    smoothed = np.convolve(mean_rewards, np.ones(window)/window, mode='valid')
    ax1.plot(range(window-1, len(mean_rewards)), smoothed, 
             label=name, color=colors[name], linewidth=3)

ax1.axhline(y=-13, color='gray', linestyle='--', alpha=0.5, label='Optimal (no cliff)')

ax1.set_xlabel('Episode', fontsize=12)
ax1.set_ylabel('Total Reward', fontsize=12)
ax1.set_title('Cliff Walking: Learning Curves\n(Lower variance = safer)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Right: Variance analysis
ax2 = axes[1]

# Calculate variance over episodes for each algorithm
variances = {}
for name, runs in results_cliff.items():
    # Variance across runs at each episode
    var_per_ep = np.var(runs, axis=0)
    # Smooth it
    variances[name] = np.convolve(var_per_ep, np.ones(window)/window, mode='valid')

for name, var in variances.items():
    ax2.plot(range(window-1, len(results_cliff[name][0])), var, 
             label=name, color=colors[name], linewidth=2)

ax2.set_xlabel('Episode', fontsize=12)
ax2.set_ylabel('Variance Across Runs', fontsize=12)
ax2.set_title('Reward Variance\n(Lower = More Consistent)', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print analysis
print("\nCLIFF WALKING ANALYSIS:")
print("-"*50)
for name in cliff_algorithms:
    final_mean = np.mean(means_cliff[name][-50:])
    final_var = np.mean([np.var(r[-50:]) for r in results_cliff[name]])
    print(f"{name:15s}: Mean = {final_mean:.1f}, Variance = {final_var:.1f}")

print("\nSARSA has LOWER variance (fewer cliff falls!)")
print("Q-Learning has HIGHER variance (occasional disasters!)")

---
## Comparison 3: Learning Speed

How quickly does each algorithm reach good performance?

In [ ]:
# Analyze learning speed

def episodes_to_threshold(rewards, threshold=-5, window=50):
    """Find how many episodes to reach threshold performance."""
    smoothed = np.convolve(rewards, np.ones(window)/window, mode='valid')
    above_threshold = np.where(smoothed > threshold)[0]
    if len(above_threshold) > 0:
        return above_threshold[0] + window
    return len(rewards)  # Never reached


print("LEARNING SPEED ANALYSIS")
print("="*60)
print("Episodes needed to reach average reward > -5 on GridWorld:\n")

speed_results = {name: [] for name in algorithms}

for name in algorithms:
    for rewards in results_grid[name]:
        eps = episodes_to_threshold(rewards)
        speed_results[name].append(eps)
    
    mean_eps = np.mean(speed_results[name])
    std_eps = np.std(speed_results[name])
    print(f"{name:15s}: {mean_eps:.0f} ± {std_eps:.0f} episodes")

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))

positions = range(len(algorithms))
means = [np.mean(speed_results[name]) for name in algorithms]
stds = [np.std(speed_results[name]) for name in algorithms]

bars = ax.bar(positions, means, yerr=stds, capsize=5, 
              color=[colors[name] for name in algorithms], 
              edgecolor='black', linewidth=2)

ax.set_xticks(positions)
ax.set_xticklabels(list(algorithms.keys()))
ax.set_ylabel('Episodes to Reach Threshold', fontsize=12)
ax.set_title('Learning Speed Comparison\n(Lower = Faster Learning)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')

# Add value labels
for i, (bar, mean) in enumerate(zip(bars, means)):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 30, 
            f'{mean:.0f}', ha='center', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

---
## The Complete Comparison Table

```
    ┌────────────────────────────────────────────────────────────────────────────┐
    │                    COMPLETE ALGORITHM COMPARISON                           │
    ├────────────────────────────────────────────────────────────────────────────┤
    │                                                                            │
    │  PROPERTY           │ MONTE CARLO  │ TD/Q-LEARNING │ SARSA                │
    │  ─────────────────────────────────────────────────────────────────────────│
    │  Update timing      │ End of ep.   │ Every step    │ Every step           │
    │  Bootstraps?        │ No           │ Yes           │ Yes                  │
    │  Bias               │ None         │ Some (init)   │ Some (init)          │
    │  Variance           │ High         │ Lower         │ Lower                │
    │  Policy type        │ On-policy    │ Off-policy    │ On-policy            │
    │  Learns             │ Q^π          │ Q*            │ Q^π                  │
    │  Continuing tasks?  │ No           │ Yes           │ Yes                  │
    │  Safety             │ N/A          │ Risky         │ Safe                 │
    │  Sample efficiency  │ Lower        │ Higher        │ Higher               │
    │                                                                            │
    │  BEST USE CASES:                                                          │
    │  ─────────────────────────────────────────────────────────────────────────│
    │  Monte Carlo        │ Blackjack, card games, episodic with clear endings  │
    │  Q-Learning         │ Video games, simulations, finding optimal policy    │
    │  SARSA              │ Robotics, trading, safety-critical applications     │
    │                                                                            │
    └────────────────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Create a visual decision tree

fig, ax = plt.subplots(figsize=(14, 10))
ax.set_xlim(0, 14)
ax.set_ylim(0, 12)
ax.axis('off')

ax.text(7, 11.5, 'How to Choose Your RL Algorithm', ha='center', fontsize=18, fontweight='bold')

# Decision node 1
box1 = FancyBboxPatch((4.5, 9), 5, 1.2, boxstyle="round,pad=0.1",
                       facecolor='#e3f2fd', edgecolor='#1976d2', linewidth=2)
ax.add_patch(box1)
ax.text(7, 9.6, 'Does your task have clear episodes?', ha='center', fontsize=11, fontweight='bold')

# Branches from node 1
ax.annotate('', xy=(3.5, 8), xytext=(5.5, 9),
            arrowprops=dict(arrowstyle='->', lw=2, color='#666'))
ax.text(4, 8.7, 'No', fontsize=10, color='#d32f2f')

ax.annotate('', xy=(10.5, 8), xytext=(8.5, 9),
            arrowprops=dict(arrowstyle='->', lw=2, color='#666'))
ax.text(10, 8.7, 'Yes', fontsize=10, color='#388e3c')

# Left branch: No episodes
box2a = FancyBboxPatch((1, 6.5), 5, 1.2, boxstyle="round,pad=0.1",
                        facecolor='#fff3e0', edgecolor='#f57c00', linewidth=2)
ax.add_patch(box2a)
ax.text(3.5, 7.1, 'Use TD-based methods!', ha='center', fontsize=11, fontweight='bold')

# Right branch: Has episodes
box2b = FancyBboxPatch((8, 6.5), 5, 1.2, boxstyle="round,pad=0.1",
                        facecolor='#e3f2fd', edgecolor='#1976d2', linewidth=2)
ax.add_patch(box2b)
ax.text(10.5, 7.1, 'Are episodes short?', ha='center', fontsize=11, fontweight='bold')

# Decision 2a: Safety question
ax.annotate('', xy=(2, 5.5), xytext=(2.5, 6.5),
            arrowprops=dict(arrowstyle='->', lw=2, color='#666'))
ax.annotate('', xy=(5, 5.5), xytext=(4.5, 6.5),
            arrowprops=dict(arrowstyle='->', lw=2, color='#666'))

box3a = FancyBboxPatch((0.5, 4), 3, 1.2, boxstyle="round,pad=0.1",
                        facecolor='#e3f2fd', edgecolor='#1976d2', linewidth=2)
ax.add_patch(box3a)
ax.text(2, 4.6, 'Safety critical?', ha='center', fontsize=10, fontweight='bold')

# Final answers from left branch
ax.annotate('', xy=(1, 3), xytext=(1.5, 4),
            arrowprops=dict(arrowstyle='->', lw=2, color='#388e3c'))
ax.text(0.8, 3.5, 'Yes', fontsize=9, color='#388e3c')

sarsa_box = FancyBboxPatch((0, 1.5), 2, 1.2, boxstyle="round,pad=0.1",
                            facecolor='#f3e5f5', edgecolor='#7b1fa2', linewidth=3)
ax.add_patch(sarsa_box)
ax.text(1, 2.1, '🛡️ SARSA', ha='center', fontsize=12, fontweight='bold', color='#7b1fa2')

ax.annotate('', xy=(3, 3), xytext=(2.5, 4),
            arrowprops=dict(arrowstyle='->', lw=2, color='#d32f2f'))
ax.text(3.2, 3.5, 'No', fontsize=9, color='#d32f2f')

ql_box = FancyBboxPatch((2.5, 1.5), 2.5, 1.2, boxstyle="round,pad=0.1",
                         facecolor='#c8e6c9', edgecolor='#388e3c', linewidth=3)
ax.add_patch(ql_box)
ax.text(3.75, 2.1, '🎯 Q-Learning', ha='center', fontsize=12, fontweight='bold', color='#388e3c')

# Right branch: Episode length
ax.annotate('', xy=(9, 5.5), xytext=(9.5, 6.5),
            arrowprops=dict(arrowstyle='->', lw=2, color='#388e3c'))
ax.text(8.8, 6, 'Yes', fontsize=9, color='#388e3c')

ax.annotate('', xy=(12, 5.5), xytext=(11.5, 6.5),
            arrowprops=dict(arrowstyle='->', lw=2, color='#d32f2f'))
ax.text(12.2, 6, 'No', fontsize=9, color='#d32f2f')

# Monte Carlo and TD from right branch
mc_box = FancyBboxPatch((7.5, 4), 3, 1.2, boxstyle="round,pad=0.1",
                         facecolor='#bbdefb', edgecolor='#1976d2', linewidth=3)
ax.add_patch(mc_box)
ax.text(9, 4.6, '🔨 Monte Carlo', ha='center', fontsize=12, fontweight='bold', color='#1976d2')

td_box = FancyBboxPatch((11, 4), 2.5, 1.2, boxstyle="round,pad=0.1",
                         facecolor='#fff3e0', edgecolor='#f57c00', linewidth=3)
ax.add_patch(td_box)
ax.text(12.25, 4.6, '⚡ TD', ha='center', fontsize=12, fontweight='bold', color='#f57c00')

plt.tight_layout()
plt.show()

---
## Summary: Key Takeaways

### The Four Algorithms

| Algorithm | Key Insight | Best For |
|-----------|------------|----------|
| **Monte Carlo** | Uses actual complete returns | Short episodic tasks, unbiased estimates |
| **TD Learning** | Bootstraps from estimates | Continuous tasks, faster learning |
| **Q-Learning** | Off-policy, learns optimal Q* | Simulations, games, finding best policy |
| **SARSA** | On-policy, accounts for exploration | Safety-critical, real-world systems |

### Key Trade-offs

| Trade-off | Left Side | Right Side |
|-----------|-----------|------------|
| **Bias vs Variance** | MC (unbiased, high var) | TD (biased, low var) |
| **Safety vs Optimality** | SARSA (safe) | Q-Learning (optimal) |
| **Episode vs Step** | MC (end of episode) | TD (every step) |

### Quick Decision Guide

1. **No clear episodes?** → Use TD/Q-Learning/SARSA
2. **Short episodes?** → Monte Carlo is fine
3. **Safety matters?** → Use SARSA
4. **Want optimal policy?** → Use Q-Learning
5. **Need fast learning?** → Use TD methods

---
## Test Your Understanding

**1. What's the main difference between MC and TD methods?**
<details>
<summary>Click to reveal answer</summary>
MC waits until the end of an episode to update using actual returns (G). TD updates every step using bootstrapped estimates (r + γV(s')). MC is unbiased but high variance; TD has some bias but lower variance and can learn faster.
</details>

**2. Why does Q-Learning find the optimal policy but SARSA doesn't always?**
<details>
<summary>Click to reveal answer</summary>
Q-Learning uses max Q(s', a') which assumes optimal future behavior. SARSA uses the actual next action which includes exploration. So Q-Learning learns Q* (optimal value) while SARSA learns Q^π (value of current ε-greedy policy).
</details>

**3. When would you choose SARSA over Q-Learning?**
<details>
<summary>Click to reveal answer</summary>
Choose SARSA when:
- Mistakes during learning are costly (robotics, trading)
- Safety is more important than finding the absolute optimal policy
- You'll continue using ε-greedy behavior after training
- The environment has "cliffs" or dangerous states
</details>

**4. Why can't Monte Carlo be used for continuing (non-episodic) tasks?**
<details>
<summary>Click to reveal answer</summary>
MC requires complete returns: G = r₁ + γr₂ + γ²r₃ + ... until episode end. In continuing tasks, there's no end, so you can never compute G. TD methods can update every step using bootstrapping, so they work for any task.
</details>

**5. If you had to pick ONE algorithm for a new RL problem, which would be the safest choice?**
<details>
<summary>Click to reveal answer</summary>
Q-Learning is often the best default choice because:
- Works for episodic and continuing tasks
- Learns the optimal policy
- Simple to implement
- Fast learning due to bootstrapping

But if safety matters, switch to SARSA. If episodes are short and you want unbiased estimates, consider MC.
</details>

---
## Congratulations!

You've completed the **Classic Algorithms** section! You now understand:

- ✅ **Monte Carlo** - Learning from complete episodes
- ✅ **TD Learning** - Bootstrapping from estimates  
- ✅ **Q-Learning** - Off-policy optimal learning
- ✅ **SARSA** - On-policy safe learning
- ✅ **How to choose** the right algorithm for your problem!

These are the foundations that ALL modern RL builds upon!

**Next Steps:**
- **[Deep RL](../deep-rl/)** - Use neural networks to scale to complex problems
- **[Policy Gradient](../policy-gradient/)** - Learn policies directly

---

*"Now you have the complete classic RL toolkit. Choose wisely!"*